# Plotting ECoG Like Movie Activity

Some references:
1.http://www.martinos.org/mne/stable/auto_tutorials/misc/plot_ecog.html?highlight=ecog

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from mayavi import mlab

import mne
from mne.viz import plot_alignment, snapshot_brain_montage

import os
import sys
import json
import scipy
import numpy as np
import collections
import warnings

warnings.filterwarnings("ignore")

import os
import sys
from mayavi import mlab
import scipy

# Append the location of img_pipe to your path if it's not already there
import sys; sys.path.append("../../../")
from img_pipe.img_pipe import img_pipe

%matplotlib inline
# %load_ext autoreload
# %autoreload 2

sys.path.append("../../../../eztrackv2/")

from eztrack.edp.objects.clinical.master_clinical import MasterClinicalSheet
from eztrack.edp.loaders.dataset.clinical.excel_meta import ExcelReader
from eztrack.edm.classifiers.model.cez_oez_analyzer import FragilitySplitAnalyzer
from eztrack.edp.loaders.dataset.result.resultloader import ResultLoader
from eztrack.base.utils.data_science_utils import *

In [2]:
class MatReader():
    '''
    Object to read mat files into a nested dictionary if need be.
    Helps keep strucutre from matlab similar to what is used in python.
    '''

    def __init__(self, filename=None):
        self.filename = filename

    def loadmat(self, filename):
        '''
        this function should be called instead of direct spio.loadmat
        as it cures the problem of not properly recovering python dictionaries
        from mat files. It calls the function check keys to cure all entries
        which are still mat-objects
        '''
        data = scipy.io.loadmat(
            filename,
            struct_as_record=False,
            squeeze_me=True)
        return self._check_keys(data)

    def _check_keys(self, dict):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in dict:
            if isinstance(dict[key], scipy.io.matlab.mio5_params.mat_struct):
                dict[key] = self._todict(dict[key])
        return dict

    def _todict(self, matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        dict = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
                dict[strg] = self._todict(elem)
            elif isinstance(elem, np.ndarray):
                dict[strg] = self._tolist(elem)
            else:
                dict[strg] = elem
        return dict

    def _tolist(self, ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, scipy.io.matlab.mio5_params.mat_struct):
                elem_list.append(self._todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(self._tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list

    def convertMatToJSON(self, matData, fileName):
        jsonData = {}

        for key in matData.keys():
            if (type(matData[key])) is np.ndarray:
                serializedData = pickle.dumps(
                    matData[key], protocol=0)  # protocol 0 is printable ASCII
                jsonData[key] = serializedData
            else:
                jsonData[key] = matData[key]

        with contextlib.closing(bz2.BZ2File(fileName, 'wb')) as f:
            json.dump(jsonData, f)

In [3]:
center = 'umf'
patid = 'umf001'
datasetid = 'sz'

# datadir = f"/home/adam2392/hdd/data/processed/output_new/trimmed/networkstatic/common_avg/ieeg/{center}/{freqband}/"
datadir = f"/Users/adam2392/Downloads/output_new/fragility/monopolar/ieeg/{center}"
filepath = os.path.join(datadir, f"{patid}_{datasetid}_frag.json")

allfiles = [f for f in os.listdir(datadir) if f.endswith(".npz")]
print(allfiles)

loader = ResultLoader(results_dir=datadir,
                      jsonfilepath=filepath,
                     datatype='frag')

['umf001_ii_asleep_fragmodel.npz', 'umf003_ii_awake_fragmodel.npz', 'umf003_sz_fragmodel.npz', 'umf004_ii_awake_fragmodel.npz', 'umf005_ii_asleep_fragmodel.npz', 'umf005_ii_awake_fragmodel.npz', 'umf002_ii_awake_fragmodel.npz', 'umf002_ii_asleep_fragmodel.npz', 'umf002_sz_fragmodel.npz', 'umf001_ii_awake_fragmodel.npz', 'umf004_ii_asleep_fragmodel.npz', 'umf005_sz_fragmodel.npz', 'umf001_sz_fragmodel.npz', 'umf004_sz_fragmodel.npz', 'umf003_ii_asleep_fragmodel.npz']


In [4]:
resultobj = loader.loadpipeline()

print(resultobj)

Loading results data from:  /Users/adam2392/Downloads/output_new/fragility/monopolar/ieeg/umf/umf001_sz_fragmodel.npz
umf001 sz Fragility Model (76, 1535)


In [5]:
fragmat = resultobj.get_data()

# Load Brain Surface(s)

In [6]:
subj_dir = '/home/adam2392/hdd/data/neuroimaging/freesurfer_output/'
subj_dir = "/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/"

fs_dir = '/opt/freesurfer/'
fs_dir = "/Users/adam2392/Documents/freesurfer/"

patient = img_pipe.freeCoG(subj=patid,
                           hem='stereo',
                           subj_dir=subj_dir,
                           fs_dir=fs_dir)

outfile = f"{patid}_elecs_all"

In [7]:
elecsdict = MatReader().loadmat(os.path.join(subj_dir, patid, "elecs", outfile))
elecmatrix = elecsdict['elecmatrix']
eleclabels = elecsdict['eleclabels']
elecanat = elecsdict['anatomy']

eleclabels = [x[0] for x in eleclabels]
elecanat = [x[3] for x in elecanat]

# print(elecanat)
# print(elecmatrix)
print(eleclabels)
print(elecsdict.keys())

['btm1', 'btm2', 'btm3', 'btm4', 'btm5', 'btm6', 'btp1', 'btp2', 'btp3', 'btp4', 'btp5', 'btp6', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64']
dict_keys(['__header__', '__version__', '__globals__', 'anatomy', 'eleclabels', 'elecmatrix'])


In [8]:
ch_names = eleclabels
dig_ch_pos = dict(zip(ch_names, elecmatrix / 1000.))
mon = mne.channels.DigMontage(dig_ch_pos=dig_ch_pos)
print('Created %s channel positions' % len(ch_names))

info = mne.create_info(ch_names, 1000., 'ecog', montage=mon)

Created 76 channel positions


# Plotting A Brain Figure / Video

In [9]:
all_elecs = patient.get_elecs(elecfile_prefix=outfile)['elecmatrix']
print(all_elecs.shape)

(76, 3)


In [10]:
precentral_elecs = patient.get_elecs(elecfile_prefix=outfile, roi='precentral')['elecmatrix']
postcentral_elecs = patient.get_elecs(elecfile_prefix=outfile, roi='postcentral')['elecmatrix']
pial = patient.get_surf(hem='rh')

In [11]:
print(patient.hem)

stereo


In [46]:
stereopial = collections.defaultdict(list)

for hem in pial.keys():
    tri = pial[hem]['tri']
    vert = pial[hem]['vert']
    
    stereopial['tri'].extend(list(tri))
    stereopial['vert'].extend(list(vert))
stereopial['tri'] = np.array(stereopial['tri'])
stereopial['vert'] = np.array(stereopial['vert'])

In [47]:
pial = stereopial
print(pial['tri'].shape)

(375622, 3)


In [12]:
from img_pipe.img_pipe.plotting.ctmr_brain_plot import ctmr_gauss_plot
from img_pipe.img_pipe.plotting.ctmr_brain_plot import el_add
from matplotlib import cm
from matplotlib import pyplot as plt

In [48]:
outputvidfile = f"/Users/adam2392/Downloads/{patid}_frag_movie.mp3"

if not os.path.exists(f"/Users/adam2392/Downloads/brainvidfigs/{patid}"):
    os.makedirs(f"/Users/adam2392/Downloads/brainvidfigs/{patid}")
    
fragfig = lambda x: os.path.join(f"/Users/adam2392/Downloads/brainvidfigs/{patid}", 
                                 f"{x}.png")

In [49]:
# %matplotlib
# %gui qt

# create mesh and 3D plot of the brain surface
mesh, mlab = ctmr_gauss_plot(tri=pial['tri'], vert=pial['vert'])

# create color on electrodes
cmap = cm.jet
all_colors = cmap(np.linspace(0,1,all_elecs.shape[0]))[:,:3]
el_add(all_elecs, color=all_colors)
# el_add(precentral_elecs, color = precentral_colors)
# el_add(postcentral_elecs, color = postcentral_colors)

if patient.hem=='lh':
    azimuth=180
elif patient.hem=='rh':
    azimuth=200
else:
    azimuth=0
print("Azimuth is: ", azimuth)
mlab.view(azimuth, elevation=90)

# start recording
start = 600
ending = fragmat.shape[1]
for i in range(start, ending):
    # create color on electrodes
    cmap = cm.jet
    all_colors = cmap(fragmat[:,i])[:,:3]
    el_add(all_elecs, color=all_colors)

    f = mlab.gcf()
    f.scene._lift()
    p = mlab.text3d(75, 75, 0, f"At time {i}", scale=5.)
    arr = mlab.screenshot(antialiased=True)
    fig, ax = plt.subplots(1,1,figsize=(20,10))
    ax.imshow(arr, aspect='equal')           
    ax.axis('off')
    mlab.savefig(fragfig(i), figure=mlab.gcf())
    p.remove()
    plt.close()
# plt.show()
print("Don't showing")

Azimuth is:  0


KeyboardInterrupt: 

In [42]:
print(resultobj.onsetwin, resultobj.offsetwin)
text = mlab.text3d(75, 75, 0, f"At time {i}", scale=5.)
print(text)

None None


In [11]:
fig = plot_alignment(info, subject=patid, 
                     subjects_dir=subj_dir,
                     surfaces=['pial'])
mlab.view(200, 70)
# plt.show()

(200.0, 70.0, 0.3373556518554688, array([0., 0., 0.]))

In [11]:
# %gui qt
# We'll once again plot the surface, then take a snapshot.
fig_scatter = plot_alignment(info, subject=patid, 
                             subjects_dir=subj_dir,
                             surfaces='pial')
mlab.view(200, 70)
f = mlab.gcf()
f.scene._lift()
xy, im = snapshot_brain_montage(fig_scatter, mon)

# Convert from a dictionary to array to plot
xy_pts = np.vstack([xy[ch] for ch in info['ch_names']])

# Define an arbitrary "activity" pattern for viz
activity = np.linspace(100, 200, xy_pts.shape[0])

# This allows us to use matplotlib to create arbitrary 2d scatterplots
_, ax = plt.subplots(figsize=(10, 10))
ax.imshow(im)
ax.scatter(*xy_pts.T, c=activity, s=200, cmap='coolwarm')
ax.set_axis_off()
plt.show()

ValueError: cannot reshape array of size 12 into shape (0,0,3)